In [4]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import scipy.odr as odr
import matplotlib
import icartt

In [3]:
#Function purpose: get new name of file
#Input:     filename: file being analyzed
#Return:    new_file_name: name of new file
def get_new_name(filename):
    campaigns = ['ACTIVATE','CAMP2EX','DISCOVER','FIREX','DC3','KORUS','SEAC4RS','NAAMES']
    instrument = 'OPTICAL'
    campaign = ""
    for x in campaigns:
        if x in filename: campaign = x
    if campaign == 'DC3':
        print('ERROR: scat and abs data are in different files')
    new_file_name = campaign + '_' + instrument + '_'
    return(new_file_name)

In [2]:
#Function purpose: I don't know
def SAE_fit_one_measurement( wavelength, SAE_fit, SAE_constant):
    return  SAE_constant*np.power(wavelength, -SAE_fit)

#Function purpose: I don't know
def AAE_fit_one_measurement( wavelength, AAE_fit, AAE_constant):
    return  AAE_constant*np.power(wavelength, -AAE_fit)

In [3]:
#Function purpose: Power fit for Scattering Angstrom Exponent, masking erroneous values (Modified Code)
#Input: bscat_wvl_red/green/blue: scatterinng values at original wavelengths
#       w1, w2, w3: original wavelengths from instrument measuring scattering
#       w_global_1/2/3: desired wavelengths
#Returns: file_M_3: dataframe containing SAE and corrected scattering values

def SAE_power_fit(bscat_wvl_red,bscat_wvl_green,bscat_wvl_blue,w1,w2,w3,w_global_1,w_global_2,w_global_3):

    if(w1 <= 0 and w2 <= 0 and w3 <= 0):
        print("WARNING: wavelength <= 0")

    wavelength_instrument=np.array([w1,w2,w3])  # modify if necessary

    temp_SAE = np.nan; temp_SAE_constant = np.nan
    temp_sigma = np.nan; temp_sigma_constant = np.nan  #uncertainty of fit parameters

    SAE_three_lambda = np.empty(len(bscat_wvl_red)); SAE_constant = np.empty(len(bscat_wvl_red))
    sigma_SAE = np.empty(len(bscat_wvl_red)); sigma_constant = np.empty(len(bscat_wvl_red))

    #hide invalid values
    mask_red = bscat_wvl_red.mask(bscat_wvl_red == 0)
    mask_green = bscat_wvl_green.mask(bscat_wvl_green == 0)
    mask_blue = bscat_wvl_blue.mask(bscat_wvl_blue == 0)

    mask_RG = (mask_red/mask_green).mask((mask_red/mask_green == 1))
    mask_RG = mask_RG.mask((mask_red/mask_green) < 0)

    mask_RB = (mask_red/mask_blue).mask((mask_red/mask_blue) == 1)
    mask_RB = mask_RB.mask((mask_red/mask_blue) < 0)

    mask_GB = (mask_green/mask_blue).mask((mask_green/mask_blue) == 1)
    mask_GB = mask_GB.mask((mask_green/mask_blue) < 0)

    #compute SAE_two_lambda, which are used later when initializing SAE_three_lambda
    SAE_R_G = -np.log(mask_RG)/np.log(wavelength_instrument[0]/wavelength_instrument[1])
    SAE_R_B = -np.log(mask_RB)/np.log(wavelength_instrument[0]/wavelength_instrument[2])
    SAE_G_B = -np.log(mask_GB)/np.log(wavelength_instrument[1]/wavelength_instrument[2])

    W_coef = np.zeros(np.size((SAE_G_B.values)))
    w_sigma = np.zeros(np.size((SAE_G_B.values)))

    for m in range(len(bscat_wvl_red)):
            if (np.isnan(bscat_wvl_red[m])==0 and np.isnan(bscat_wvl_green[m])==0 and np.isnan(bscat_wvl_blue[m])==0):

                one_babs_at_three_lambda=[bscat_wvl_red[m],bscat_wvl_green[m],bscat_wvl_blue[m]]
                
                #use SAE_G_B as initial input 
                if((np.power(wavelength_instrument[1], -SAE_G_B.values[m])) > 0):
                    W_coef = np.array([SAE_G_B.values[m], bscat_wvl_green[m]/ (np.power(wavelength_instrument[1], -SAE_G_B.values[m]))] )
                    try:
                        W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                        w_sigma = np.sqrt(np.diag(w_sigma))
                    except RuntimeError:
                        pass
                temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                #use SAE_R_G as initial input
                if(((np.power(wavelength_instrument[0], -SAE_R_G.values[m]))) > 0 ):
                    W_coef = np.array([SAE_R_G.values[m], bscat_wvl_red[m]/ (np.power(wavelength_instrument[0], -SAE_R_G.values[m]))]  ) 
                    try:          
                        W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                        w_sigma = np.sqrt(np.diag(w_sigma))
                    except RuntimeError:
                        pass    #do not replace the previous results

                #make sure w_sigma exists
                if(w_sigma[0] != -1):
                    #check if the current inital guess (SAE_R_G) is better than the previous one (SAE_G_B): ? w_sigma is smaller
                    if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                
                        temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                    else:
                        pass       #do not replace the previous results
            
                #use SAE_R_B as initial input
                if((np.power(wavelength_instrument[2], -SAE_R_B.values[m])) > 0):
                    W_coef = np.array([SAE_R_B.values[m],bscat_wvl_blue[m]/ (np.power(wavelength_instrument[2], -SAE_R_B.values[m]))])
                    try:   
                        W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                        w_sigma = np.sqrt(np.diag(w_sigma))
                    except RuntimeError:
                        pass
                #make sure w_sigma exists
                if(w_sigma[0] != -1):               
                    #check if the current initial guess (SAE_R_B) is better than the previous one (SAE_G_B or SAE_R_G): ? w_sigma is smaller
                    if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                        
                        temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                    else: 
                        pass #do not replace the previous results   
            
            #Output the results
            SAE_three_lambda[m]=temp_SAE; SAE_constant[m]=temp_SAE_constant
            sigma_SAE[m]=temp_sigma; sigma_constant[m]=temp_sigma_constant
            #clear temp_variables
            temp_SAE=np.nan;temp_SAE_constant=np.nan
            temp_sigma=np.nan;temp_sigma_constant=np.nan

    b_scat_red=SAE_constant*(np.power(w_global_1, -SAE_three_lambda))
    b_scat_green=SAE_constant*(np.power(w_global_2, -SAE_three_lambda))
    b_scat_blue=SAE_constant*(np.power(w_global_3, -SAE_three_lambda))

    file_M_3_Bscat = pd.DataFrame() 
    file_M_3_Bscat['SAE'] = SAE_three_lambda; 
    file_M_3_Bscat['b_scat_red'] = b_scat_red ;file_M_3_Bscat['b_scat_green'] = b_scat_green; file_M_3_Bscat['b_scat_blue'] = b_scat_blue ; 

    return file_M_3_Bscat

In [4]:
#Function purpose: Power fit for Scattering Angstrom Exponent, without masking erroneous values (Original Code)
#Input: bscat_wvl_red/green/blue: scatterinng values at original wavelengths
#       w1, w2, w3: original wavelengths from instrument measuring scattering
#       w_global_1/2/3: desired wavelengths
#Returns: file_M_3: dataframe containing SAE and corrected scattering values

def SAE_power_fit_2(bscat_wvl_red,bscat_wvl_green,bscat_wvl_blue,w1,w2,w3,w_global_1,w_global_2,w_global_3):

    if(w1 <= 0 and w2 <= 0 and w3 <= 0):
        print("WARNING: wavelength <= 0")

    wavelength_instrument=np.array([w1,w2,w3])  # modify if necessary

    temp_SAE = np.nan; temp_SAE_constant = np.nan
    temp_sigma = np.nan; temp_sigma_constant = np.nan  #uncertainty of fit parameters

    SAE_three_lambda = np.empty(len(bscat_wvl_red)); SAE_constant = np.empty(len(bscat_wvl_red))
    sigma_SAE = np.empty(len(bscat_wvl_red)); sigma_constant = np.empty(len(bscat_wvl_red))

    #compute SAE_two_lambda, which are used later when initializing SAE_three_lambda
    SAE_R_G = -np.log(bscat_wvl_red/bscat_wvl_green)/np.log(wavelength_instrument[0]/wavelength_instrument[1])
    SAE_R_B = -np.log(bscat_wvl_red/bscat_wvl_blue)/np.log(wavelength_instrument[0]/wavelength_instrument[2])
    SAE_G_B = -np.log(bscat_wvl_green/bscat_wvl_blue)/np.log(wavelength_instrument[1]/wavelength_instrument[2])

    W_coef = np.zeros(np.size((SAE_G_B.values)))
    w_sigma = np.zeros(np.size((SAE_G_B.values)))

    for m in range(len(bscat_wvl_red)):
            if (np.isnan(bscat_wvl_red.iloc[m])==0 and np.isnan(bscat_wvl_green.iloc[m])==0 and np.isnan(bscat_wvl_blue.iloc[m])==0):

                one_babs_at_three_lambda=[bscat_wvl_red.iloc[m],bscat_wvl_green.iloc[m],bscat_wvl_blue.iloc[m]]
                
                #use SAE_G_B as initial input 
                if((np.power(wavelength_instrument[1], -SAE_G_B.values[m])) > 0):
                    W_coef = np.array([SAE_G_B.values[m], bscat_wvl_green.iloc[m]/ (np.power(wavelength_instrument[1], -SAE_G_B.values[m]))] )
                    try:
                        W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                        w_sigma = np.sqrt(np.diag(w_sigma))
                    except RuntimeError:
                        pass
                temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                #use SAE_R_G as initial input
                if(((np.power(wavelength_instrument[0], -SAE_R_G.values[m]))) > 0 ):
                    W_coef = np.array([SAE_R_G.values[m], bscat_wvl_red.iloc[m]/ (np.power(wavelength_instrument[0], -SAE_R_G.values[m]))]  ) 
                    try:          
                        W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                        w_sigma = np.sqrt(np.diag(w_sigma))
                    except RuntimeError:
                        pass    #do not replace the previous results

                #make sure w_sigma exists
                if(w_sigma[0] != -1):
                    #check if the current inital guess (SAE_R_G) is better than the previous one (SAE_G_B): ? w_sigma is smaller
                    if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                
                        temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                    else:
                        pass       #do not replace the previous results
            
                #use SAE_R_B as initial input
                if((np.power(wavelength_instrument[2], -SAE_R_B.values[m])) > 0):
                    W_coef = np.array([SAE_R_B.values[m],bscat_wvl_blue.iloc[m]/ (np.power(wavelength_instrument[2], -SAE_R_B.values[m]))])
                    try:   
                        W_coef, w_sigma = curve_fit(SAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                        w_sigma = np.sqrt(np.diag(w_sigma))
                    except RuntimeError:
                        pass
                #make sure w_sigma exists
                if(w_sigma[0] != -1):               
                    #check if the current initial guess (SAE_R_B) is better than the previous one (SAE_G_B or SAE_R_G): ? w_sigma is smaller
                    if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                        
                        temp_SAE = W_coef[0];temp_SAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
                    else: 
                        pass #do not replace the previous results   
            
            #Output the results
            SAE_three_lambda[m]=temp_SAE; SAE_constant[m]=temp_SAE_constant
            sigma_SAE[m]=temp_sigma; sigma_constant[m]=temp_sigma_constant
            #clear temp_variables
            temp_SAE=np.nan;temp_SAE_constant=np.nan
            temp_sigma=np.nan;temp_sigma_constant=np.nan

    b_scat_red=SAE_constant*(np.power(w_global_1, -SAE_three_lambda))
    b_scat_green=SAE_constant*(np.power(w_global_2, -SAE_three_lambda))
    b_scat_blue=SAE_constant*(np.power(w_global_3, -SAE_three_lambda))

    file_M_3_Bscat = pd.DataFrame() 
    file_M_3_Bscat['SAE'] = SAE_three_lambda; 
    file_M_3_Bscat['b_scat_red'] = b_scat_red ;file_M_3_Bscat['b_scat_green'] = b_scat_green; file_M_3_Bscat['b_scat_blue'] = b_scat_blue ; 

    return file_M_3_Bscat

In [5]:
#Function purpose: Power fit for Absorption Angstrom Exponent, masking erroneous values (Modified Code)
#Input: bsabs_wvl_red/green/blue: absorption values at original wavelengths
#       w1, w2, w3: original wavelengths from instrument measuring absorption
#       w_global_1/2/3: desired wavelengths
#Returns: file_M_3: dataframe containing AAE and corrected absorption values

def AAE_power_fit(babs_wvl_red,babs_wvl_green,babs_wvl_blue,w1,w2,w3,w_global_1,w_global_2,w_global_3):

    if(w1 <= 0 and w2 <= 0 and w3 <= 0):
        print("WARNING: wavelength <= 0")

    wavelength_instrument=np.array([w1,w2,w3])  # modify if necessary

    temp_AAE = np.nan; temp_AAE_constant = np.nan
    temp_sigma = np.nan; temp_sigma_constant = np.nan  #uncertainty of fit parameters

    AAE_three_lambda = np.empty(len(babs_wvl_red)); AAE_constant = np.empty(len(babs_wvl_red))
    sigma_AAE = np.empty(len(babs_wvl_red)); sigma_constant = np.empty(len(babs_wvl_red))

    #hide invalid values
    mask_red = babs_wvl_red.mask(babs_wvl_red == 0)
    mask_green = babs_wvl_green.mask(babs_wvl_green == 0)
    mask_blue = babs_wvl_blue.mask(babs_wvl_blue == 0)

    mask_RG = (mask_red/mask_green).mask((mask_red/mask_green == 1))
    mask_RG = mask_RG.mask((mask_red/mask_green) < 0)

    mask_RB = (mask_red/mask_blue).mask((mask_red/mask_blue) == 1)
    mask_RB = mask_RB.mask((mask_red/mask_blue) < 0)

    mask_GB = (mask_green/mask_blue).mask((mask_green/mask_blue) == 1)
    mask_GB = mask_GB.mask((mask_green/mask_blue) < 0)

    #compute AAE_two_lambda, which are used later when initializing AAE_three_lambda
    AAE_R_G = -np.log(mask_RG)/np.log(wavelength_instrument[0]/wavelength_instrument[1])
    AAE_R_B = -np.log(mask_RB)/np.log(wavelength_instrument[0]/wavelength_instrument[2])
    AAE_G_B = -np.log(mask_GB)/np.log(wavelength_instrument[1]/wavelength_instrument[2])

    for m in range(len(babs_wvl_red)):
        if (np.isnan(babs_wvl_red[m])==0 and np.isnan(babs_wvl_green[m])==0 and np.isnan(babs_wvl_blue[m])==0):

            one_babs_at_three_lambda=[babs_wvl_red[m],babs_wvl_green[m],babs_wvl_blue[m]]
                
            #use AAE_G_B as initial input
            W_coef = np.array([AAE_G_B.values[m], babs_wvl_green[m]/ (np.power(wavelength_instrument[1], -AAE_G_B.values[m]))] )
            try:
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                w_sigma = np.sqrt(np.diag(w_sigma))
            except RuntimeError:
                pass
            temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            #use AAE_R_G as initial input
            W_coef = np.array([AAE_R_G.values[m], babs_wvl_red[m]/ (np.power(wavelength_instrument[0], -AAE_R_G.values[m]))]  ) 
            try:          
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                w_sigma = np.sqrt(np.diag(w_sigma))
            except RuntimeError:
                pass
            
            #check if the current inital guess (AAE_R_G) is better than the previous one (AAE_G_B): ? w_sigma is smaller
            if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                
                temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            else:
                pass       #do not replace the previous results
            
            #use AAE_R_B as initial input
            W_coef = np.array([AAE_R_B.values[m],babs_wvl_blue[m]/ (np.power(wavelength_instrument[2], -AAE_R_B.values[m]))])
            try:   
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                w_sigma = np.sqrt(np.diag(w_sigma))
            except RuntimeError:
                pass               
            #check if the current initial guess (AAE_R_B) is better than the previous one (AAE_G_B or AAE_R_G): ? w_sigma is smaller
            if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                        
                temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            else: 
                pass #do not replace the previous results   
            
        #Output the results
        AAE_three_lambda[m]=temp_AAE; AAE_constant[m]=temp_AAE_constant
        sigma_AAE[m]=temp_sigma; sigma_constant[m]=temp_sigma_constant
        #clear temp_variables
        temp_AAE=np.nan;temp_AAE_constant=np.nan
        temp_sigma=np.nan;temp_sigma_constant=np.nan

    b_abs_red=AAE_constant*(np.power(w_global_1, -AAE_three_lambda))
    b_abs_green=AAE_constant*(np.power(w_global_2, -AAE_three_lambda))
    b_abs_blue=AAE_constant*(np.power(w_global_3, -AAE_three_lambda))


    file_M_3_Babs = pd.DataFrame() 
    file_M_3_Babs['AAE'] = AAE_three_lambda; 
    file_M_3_Babs['b_abs_red'] = b_abs_red ;file_M_3_Babs['b_abs_green'] = b_abs_green; file_M_3_Babs['b_abs_blue'] = b_abs_blue ; 
    
    return file_M_3_Babs

In [60]:
#Function purpose: Power fit for Absorption Angstrom Exponent, masking erroneous values (Origi Code)
#Input: bsabs_wvl_red/green/blue: absorption values at original wavelengths
#       w1, w2, w3: original wavelengths from instrument measuring absorption
#       w_global_1/2/3: desired wavelengths
#Returns: file_M_3: dataframe containing AAE and corrected absorption values

def AAE_power_fit_2(babs_wvl_red,babs_wvl_green,babs_wvl_blue,w1,w2,w3,w_global_1,w_global_2,w_global_3):

    if(w1 <= 0 and w2 <= 0 and w3 <= 0):
        print("WARNING: wavelength <= 0")

    wavelength_instrument=np.array([w1,w2,w3])  # modify if necessary

    temp_AAE = np.nan; temp_AAE_constant = np.nan
    temp_sigma = np.nan; temp_sigma_constant = np.nan  #uncertainty of fit parameters

    AAE_three_lambda = np.empty(len(babs_wvl_red)); AAE_constant = np.empty(len(babs_wvl_red))
    sigma_AAE = np.empty(len(babs_wvl_red)); sigma_constant = np.empty(len(babs_wvl_red))

    #compute AAE_two_lambda, which are used later when initializing AAE_three_lambda
    AAE_R_G = -np.log(babs_wvl_red/babs_wvl_green)/np.log(wavelength_instrument[0]/wavelength_instrument[1])
    AAE_R_B = -np.log(babs_wvl_red/babs_wvl_blue)/np.log(wavelength_instrument[0]/wavelength_instrument[2])
    AAE_G_B = -np.log(babs_wvl_green/babs_wvl_blue)/np.log(wavelength_instrument[1]/wavelength_instrument[2])

    for m in range(len(babs_wvl_red)):
        if (np.isnan(babs_wvl_red.iloc[m])==0 and np.isnan(babs_wvl_green.iloc[m])==0 and np.isnan(babs_wvl_blue.iloc[m])==0):

            one_babs_at_three_lambda=[babs_wvl_red.iloc[m],babs_wvl_green.iloc[m],babs_wvl_blue.iloc[m]]
                
            #use AAE_G_B as initial input
            W_coef = np.array([AAE_G_B.values[m], babs_wvl_green.iloc[m]/ (np.power(wavelength_instrument[1], -AAE_G_B.values[m]))] )
            try:
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                print(w_sigma)
                w_sigma = np.sqrt(np.diag(w_sigma))
                print(w_sigma)
            except RuntimeError:
                pass
            temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            #use AAE_R_G as initial input
            W_coef = np.array([AAE_R_G.values[m], babs_wvl_red.iloc[m]/ (np.power(wavelength_instrument[0], -AAE_R_G.values[m]))]  ) 
            try:          
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                print(w_sigma)
                w_sigma = np.sqrt(np.diag(w_sigma))
                print(w_sigma)
            except RuntimeError:
                pass
            
            #check if the current inital guess (AAE_R_G) is better than the previous one (AAE_G_B): ? w_sigma is smaller
            if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                
                temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            else:
                pass       #do not replace the previous results
            
            #use AAE_R_B as initial input
            W_coef = np.array([AAE_R_B.values[m],babs_wvl_blue.iloc[m]/ (np.power(wavelength_instrument[2], -AAE_R_B.values[m]))])
            try:   
                W_coef, w_sigma = curve_fit(AAE_fit_one_measurement, wavelength_instrument, one_babs_at_three_lambda,p0=W_coef, maxfev=1000)
                w_sigma = np.sqrt(np.diag(w_sigma))
            except RuntimeError:
                pass               
            #check if the current initial guess (AAE_R_B) is better than the previous one (AAE_G_B or AAE_R_G): ? w_sigma is smaller
            if (w_sigma[0]<temp_sigma):  #Yes, the current w_sigma is smaller                        
                temp_AAE = W_coef[0];temp_AAE_constant = W_coef[1];temp_sigma= w_sigma[0];temp_sigma_constant = w_sigma[1]
            else: 
                pass #do not replace the previous results   
            
        #Output the results
        AAE_three_lambda[m]=temp_AAE; AAE_constant[m]=temp_AAE_constant
        sigma_AAE[m]=temp_sigma; sigma_constant[m]=temp_sigma_constant
        #clear temp_variables
        temp_AAE=np.nan;temp_AAE_constant=np.nan
        temp_sigma=np.nan;temp_sigma_constant=np.nan

    b_abs_red=AAE_constant*(np.power(w_global_1, -AAE_three_lambda))
    b_abs_green=AAE_constant*(np.power(w_global_2, -AAE_three_lambda))
    b_abs_blue=AAE_constant*(np.power(w_global_3, -AAE_three_lambda))


    file_M_3_Babs = pd.DataFrame() 
    file_M_3_Babs['AAE'] = AAE_three_lambda; 
    file_M_3_Babs['b_abs_red'] = b_abs_red ;file_M_3_Babs['b_abs_green'] = b_abs_green; file_M_3_Babs['b_abs_blue'] = b_abs_blue ; 
    
    return file_M_3_Babs

In [ ]:
#Function purpose: get units from file
#Input:     varnames: list of the variable names
#           ict: icartt object we are analyzing
#Returns:   varDf: dataframe containing information about the variable names

def get_unit_info(varnames,ict):
    varDf = pd.DataFrame(columns=['var_name','unit','descr','descr_long'])

    #split rows of data and put into df
    for x in varnames:
        var = str(ict.variables[x])
        varAr = var.split(',')
        while len(varAr) < 4:
            varAr.append('')
        varDf.loc[len(varDf)] = varAr
    
    return(varDf)

In [54]:
#Function purpose: extract wavelength
#Input: str, the string to extract wavelength from
#Return: 
def get_wavelength(str):
    num = ""
    for s in str: 
        if s.isdigit() : num += (s)
    return int(num)

In [21]:
#MAIN: FOR ONE FILE
#load in ict file to python
filename = 'C:/Users/cphal/OneDrive/Desktop/MAC prediction model/FIREXAQ-LARGE-OPTICAL_DC8_20190716_R0.ict'
ict = icartt.Dataset(filename)
df = ict.data[:]
df = pd.DataFrame(df)

#get variable names and create dataframe based on this
varnames = [x for x in ict.variables]
df = pd.DataFrame(df, columns = varnames)

#get info about variables
#FIXME: not sure why this says it's not defined??
vardf = get_unit_info(varnames,ict)

#get scattering 
scat_mask = [('scattering at' in x) and ('Derived' not in x) for x in vardf['descr_long']]
scat_varnames = vardf['var_name'][scat_mask]

#get absorption
abs_mask = [('absorption at' in x) and ('research' not in x) for x in vardf['descr_long']]
abs_varnames = vardf['var_name'][abs_mask]

#if either of those is not exactly 3 long, there was an error, revisiting is necessary so terminate code
if(len(scat_varnames)!= 3 or len(abs_varnames)!=3):
    print('FATAL ERROR! More or less than 3 wavelengths found!')
    exit()

In [74]:
#rename scat and abs wavelengths
optical_df = pd.DataFrame()
#FIXME: use the variable renaming for this

In [ ]:
#replace values that could cause calculation errors with another flag (-3333)
optical_df.loc[optical_df['abs_red']+optical_df['scat_red'] == 0, 'abs_red'] = -3333
optical_df.loc[optical_df['abs_red']== 0, 'scat_red'] = -3333
optical_df.loc[optical_df['abs_green']+optical_df['scat_green'] == 0, 'abs_green'] = -3333
optical_df.loc[optical_df['abs_green']== 0, 'scat_green'] = -3333
optical_df.loc[optical_df['abs_blue']+optical_df['scat_blue'] == 0, 'abs_blue'] = -3333
optical_df.loc[optical_df['abs_blue']== 0, 'scat_blue'] = -3333

#calculate SSA
optical_df['SSA_red'] = optical_df.apply(lambda x: x['scat_red'] / (x['scat_red'] + x['abs_red']), axis=1)
optical_df['SSA_blue'] = optical_df.apply(lambda x: x['scat_blue'] / (x['scat_blue'] + x['abs_blue']), axis=1)
optical_df['SSA_green'] = optical_df.apply(lambda x: x['scat_green'] / (x['scat_green'] + x['abs_green']), axis=1)

#replace values where SSA <0 or >1 with a flag (-2222)
optical_df.loc[optical_df['SSA_red'] < 0, 'SSA_red'] = -2222
optical_df.loc[optical_df['SSA_green'] < 0, 'SSA_green'] = -2222
optical_df.loc[optical_df['SSA_blue'] < 0, 'SSA_blue'] = -2222
optical_df.loc[optical_df['SSA_red'] > 1, 'SSA_red'] = -2222
optical_df.loc[optical_df['SSA_green'] > 1, 'SSA_green'] = -2222
optical_df.loc[optical_df['SSA_blue'] > 1, 'SSA_blue'] = -2222

#FIXME do not run this if both abs and scat

In [49]:
#calculate ratio of abs to scat
optical_df['red_scat_abs'] = optical_df['scat_red']/optical_df['abs_red']
optical_df['green_scat_abs'] = optical_df['scat_green']/optical_df['abs_green']
optical_df['blue_scat_abs'] = optical_df['scat_blue']/optical_df['abs_blue']

#replace values where ratio < 0 with a flag (-2222)
optical_df.loc[optical_df['red_scat_abs'] < 0, 'red_scat_abs'] = -2222
optical_df.loc[optical_df['green_scat_abs'] < 0, 'green_scat_abs'] = -2222
optical_df.loc[optical_df['blue_scat_abs'] < 0, 'blue_scat_abs'] = -2222

In [78]:
#create new dataframe with no rows with -8888
#new_df = optical_df[optical_df['scat_red'] != -8888]
#new_df = new_df[new_df['scat_green'] != -8888]
#new_df = new_df[new_df['scat_blue'] != -8888]
new_df = optical_df[optical_df['abs_red'] != -8888]
new_df = new_df[new_df['abs_green'] != -8888]
new_df = new_df[new_df['abs_blue'] != -8888]

new_df.to_csv('output.csv')